In [1]:
import pandas as pd 
import numpy as np
from skimage import io
from keras.models import Sequential
from keras.layers import Dense
from keras.losses import categorical_crossentropy
from skimage.transform import rescale, resize

Using TensorFlow backend.


In [2]:
from sklearn.preprocessing import StandardScaler
def unpack(c):
    #here c is the input image in the form of a 3 dimensional array
    c_ = resize(c,(200,200,3))
    r = c_[:,:,0]
    g = c_[:,:,1]
    b = c_[:,:,2]
    #we have now extracted the rgb values of the image inputted -> to flatten the 2D array and concatenate into a single img array
    
    r2 = r.reshape(r.shape[0]*r.shape[1])
    g2 = g.reshape(g.shape[0]*g.shape[1])
    b2 = b.reshape(b.shape[0]*b.shape[1])
    temp = np.concatenate([r2, g2,b2], axis = 0)
    img_array= temp / temp.max(axis=0)
    
    return img_array

In [3]:
from PIL import Image
import glob


#running inputs read to script : 
image_list_run = []
path1 = 'C:\\Users\\Agrim Nautiyal\\Desktop\\walk-or-run-data\\train\\run\\*.png'
for p in glob.glob(path1):
    image_list_run.append(io.imread(p))
    
    

#walking inputs read to script 
image_list_walk=[]
path2 = 'C:\\Users\\Agrim Nautiyal\\Desktop\\walk-or-run-data\\train\\walk\\*.png'
for p in glob.glob(path2):
    image_list_walk.append(io.imread(p))

    
    
#to set labels for training images 
y1=[0 for i in range (len(image_list_run))]
y2=[1 for i in range (len(image_list_walk))]
y = y1+y2
#0 denotes run, 1 denotes walk 


In [4]:
x = image_list_run + image_list_walk
x_train = []
for i in x:
    x_temp = unpack(i)
    x_train.append(x_temp)

C:\Users\Agrim Nautiyal\Anaconda3\envs\dsenv\lib\site-packages\skimage\transform\_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Users\Agrim Nautiyal\Anaconda3\envs\dsenv\lib\site-packages\skimage\transform\_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


In [5]:
x_train = np.array(x_train)
sc = StandardScaler()
sc.fit(x_train)
x_train = sc.transform(x_train)

In [6]:
l =[[0 for i in range(2)] for j in range(len(y))]

for i in range(len(y)):
    j = y[i]
    #print(j)
    l[i][j] = 1
y_train = np.array(l)


In [7]:
model = Sequential()
model.add(Dense(1000, activation = 'relu', input_shape = (120000,)))
model.add(Dense(500, activation= 'relu'))
model.add(Dense(400, activation ='relu'))
model.add(Dense(200, activation = 'relu'))
model.add(Dense(100, activation =  'relu'))
model.add(Dense(2, activation='softmax'))

In [14]:
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train, epochs = 4, batch_size = 3)

Epoch 1/4
600/600 [==============================] - 766s 1s/step - loss: 8.0894 - acc: 0.4967
Epoch 2/4
120/600 [=====>........................] - ETA: 10:29 - loss: 8.4620 - acc: 0.4750

KeyboardInterrupt: 

In [35]:
test_image = io.imread("C:\\Users\\Agrim Nautiyal\\Desktop\\walk-or-run-data\\train\\run\\run_6f2b4633.png")
test_image_process = unpack(test_image)
sc.fit(test_image_process.reshape(-1,1))
test_image_process = sc.transform(test_image_process.reshape(-1,1))
y_test = model.predict(test_image_process.reshape(1,120000))
test_image_process.shape

C:\Users\Agrim Nautiyal\Anaconda3\envs\dsenv\lib\site-packages\skimage\transform\_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Users\Agrim Nautiyal\Anaconda3\envs\dsenv\lib\site-packages\skimage\transform\_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


(120000, 1)

In [38]:
res =[]
for i in x_train : 
    res.append(model.predict(i.reshape(1,120000)))